In [8]:
#Correr solo una vez
#Para bajar los stopwords
#NO CORRER! TAMPOCO COLOCARLO EN LA PARTE DE PYTHON, ESTO ES PARTE DEL ENVIROMENT LDA
#Luego en la ventana que aparece, se va a CORPORA y luego en STOPWORDS Y LUEGO DOWNLOAD
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [38]:
import re
import pickle

In [12]:
import numpy as np
import pandas as pd

In [13]:
from pprint import pprint

In [14]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [15]:
from nltk.corpus import stopwords

In [16]:
import spacy

In [17]:
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [19]:
#visualizando los stop words, que son palabras que no agregan mucho a el aprendizaje
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [20]:
#cargando el archivo de datos
df = pd.read_csv('../data/raw/reviews_all.csv')
df.head()

,rating,review
0,50,This is one of the best hotels I've ever staye...
1,50,Everything about this hotel was awesome. The s...
2,50,Our tour group stayed here for two nights. Th...
3,50,Excellent service at Porta Hotel Antigua. From...
4,50,I almost always stay at Hotel Antigua when I t...


In [21]:
# Convertir a una lista
data = df.review.values.tolist()
pprint(data[:1])

["This is one of the best hotels I've ever stayed at with incredible staff "
 'catering to your every need. I felt like family whilst I was there!   I '
 'stayed for 3 weeks during the pandemic and was so impressed with all the '
 'measures the hotel have put in place to keep everyone safe and well. They '
 'give you a box of masks, gloves and a bottle of hand sanitiser upon arrival, '
 'all the staff wear masks and gloves and the rooms and everything else are '
 "cleaned to the highest standard.  Plus the prices they've been offering "
 'during this time are incredible value for money! Definitely would recommend '
 'and hope to be back once travel reopens.   Thank you so much for looking '
 'after me so well until I could return home!']


In [22]:
# Eliminar emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
pprint(data[:1])

["This is one of the best hotels I've ever stayed at with incredible staff "
 'catering to your every need. I felt like family whilst I was there!   I '
 'stayed for 3 weeks during the pandemic and was so impressed with all the '
 'measures the hotel have put in place to keep everyone safe and well. They '
 'give you a box of masks, gloves and a bottle of hand sanitiser upon arrival, '
 'all the staff wear masks and gloves and the rooms and everything else are '
 "cleaned to the highest standard.  Plus the prices they've been offering "
 'during this time are incredible value for money! Definitely would recommend '
 'and hope to be back once travel reopens.   Thank you so much for looking '
 'after me so well until I could return home!']


In [23]:
# Eliminar newlines
data = [re.sub(r'\s+', ' ', sent) for sent in data]
pprint(data[:1])

["This is one of the best hotels I've ever stayed at with incredible staff "
 'catering to your every need. I felt like family whilst I was there! I stayed '
 'for 3 weeks during the pandemic and was so impressed with all the measures '
 'the hotel have put in place to keep everyone safe and well. They give you a '
 'box of masks, gloves and a bottle of hand sanitiser upon arrival, all the '
 'staff wear masks and gloves and the rooms and everything else are cleaned to '
 "the highest standard. Plus the prices they've been offering during this time "
 'are incredible value for money! Definitely would recommend and hope to be '
 'back once travel reopens. Thank you so much for looking after me so well '
 'until I could return home!']


In [24]:
# Eliminar comillas
data = [re.sub(r"\'", "", sent) for sent in data]
pprint(data[:1])

['This is one of the best hotels Ive ever stayed at with incredible staff '
 'catering to your every need. I felt like family whilst I was there! I stayed '
 'for 3 weeks during the pandemic and was so impressed with all the measures '
 'the hotel have put in place to keep everyone safe and well. They give you a '
 'box of masks, gloves and a bottle of hand sanitiser upon arrival, all the '
 'staff wear masks and gloves and the rooms and everything else are cleaned to '
 'the highest standard. Plus the prices theyve been offering during this time '
 'are incredible value for money! Definitely would recommend and hope to be '
 'back once travel reopens. Thank you so much for looking after me so well '
 'until I could return home!']


In [25]:
#definiendo función para tokenizar
def sent_to_words(sentences):
    for sentence in sentences:
        # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True elimina la puntuación

data_words = list(sent_to_words(data))

print(data_words[:1])

[['this', 'is', 'one', 'of', 'the', 'best', 'hotels', 'ive', 'ever', 'stayed', 'at', 'with', 'incredible', 'staff', 'catering', 'to', 'your', 'every', 'need', 'felt', 'like', 'family', 'whilst', 'was', 'there', 'stayed', 'for', 'weeks', 'during', 'the', 'pandemic', 'and', 'was', 'so', 'impressed', 'with', 'all', 'the', 'measures', 'the', 'hotel', 'have', 'put', 'in', 'place', 'to', 'keep', 'everyone', 'safe', 'and', 'well', 'they', 'give', 'you', 'box', 'of', 'masks', 'gloves', 'and', 'bottle', 'of', 'hand', 'sanitiser', 'upon', 'arrival', 'all', 'the', 'staff', 'wear', 'masks', 'and', 'gloves', 'and', 'the', 'rooms', 'and', 'everything', 'else', 'are', 'cleaned', 'to', 'the', 'highest', 'standard', 'plus', 'the', 'prices', 'theyve', 'been', 'offering', 'during', 'this', 'time', 'are', 'incredible', 'value', 'for', 'money', 'definitely', 'would', 'recommend', 'and', 'hope', 'to', 'be', 'back', 'once', 'travel', 'reopens', 'thank', 'you', 'so', 'much', 'for', 'looking', 'after', 'me', '

In [26]:
# Construimos modelos de bigrams y trigrams
# https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Aplicamos el conjunto de bigrams/trigrams a nuestros documentos
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [27]:
print(bigram_mod[data_words[0]])
print(trigram_mod[bigram_mod[data_words[0]]])

['this', 'is', 'one', 'of', 'the', 'best', 'hotels', 'ive', 'ever', 'stayed', 'at', 'with', 'incredible', 'staff', 'catering', 'to', 'your', 'every', 'need', 'felt', 'like', 'family', 'whilst', 'was', 'there', 'stayed', 'for', 'weeks', 'during', 'the', 'pandemic', 'and', 'was', 'so', 'impressed', 'with', 'all', 'the', 'measures', 'the', 'hotel', 'have', 'put', 'in', 'place', 'to', 'keep', 'everyone', 'safe', 'and', 'well', 'they', 'give', 'you', 'box', 'of', 'masks', 'gloves', 'and', 'bottle', 'of', 'hand', 'sanitiser', 'upon_arrival', 'all', 'the', 'staff', 'wear', 'masks', 'and', 'gloves', 'and', 'the', 'rooms', 'and', 'everything', 'else', 'are', 'cleaned', 'to', 'the', 'highest', 'standard', 'plus', 'the', 'prices', 'theyve', 'been', 'offering', 'during', 'this', 'time', 'are', 'incredible', 'value', 'for', 'money', 'definitely', 'would', 'recommend', 'and', 'hope', 'to', 'be', 'back', 'once', 'travel', 'reopens', 'thank', 'you', 'so', 'much', 'for', 'looking', 'after', 'me', 'so',

In [28]:
# Eliminar stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Hacer bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Hacer trigrams
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Lematización basada en el modelo de POS de Spacy
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [29]:
# Eliminamos stopwords
data_words_nostops = remove_stopwords(data_words)

In [30]:
# Formamos bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [31]:
# python3 -m spacy download en_core_web_lg
# Inicializamos el modelo 'en_core_web_lg' con las componentes de POS únicamente
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

# Lematizamos preservando únicamente noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['good', 'hotel', 'have', 'ever', 'stay', 'incredible', 'staff', 'catering', 'need', 'feel', 'family', 'stay', 'week', 'pandemic', 'impress', 'measure', 'hotel', 'put', 'place', 'keep', 'safe', 'well', 'give', 'box', 'mask', 'glove', 'bottle', 'hand', 'sanitiser', 'upon_arrival', 'staff', 'wear', 'mask', 'glove', 'room', 'else', 'clean', 'high', 'standard', 'price', 'have', 'offer', 'time', 'incredible', 'value', 'money', 'definitely', 'would', 'recommend', 'hope', 'back', 'travel', 'reopen', 'thank', 'much', 'look', 'well', 'could', 'return', 'home']]


In [33]:
# Eliminamos stopwords
data_words_nostops = remove_stopwords(data_words)
# Formamos bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

# Lematizamos preservando únicamente noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

    
# Creamos diccionario
id2word = corpora.Dictionary(data_lemmatized)


[['good', 'hotel', 'have', 'ever', 'stay', 'incredible', 'staff', 'catering', 'need', 'feel', 'family', 'stay', 'week', 'pandemic', 'impress', 'measure', 'hotel', 'put', 'place', 'keep', 'safe', 'well', 'give', 'box', 'mask', 'glove', 'bottle', 'hand', 'sanitiser', 'upon_arrival', 'staff', 'wear', 'mask', 'glove', 'room', 'else', 'clean', 'high', 'standard', 'price', 'have', 'offer', 'time', 'incredible', 'value', 'money', 'definitely', 'would', 'recommend', 'hope', 'back', 'travel', 'reopen', 'thank', 'much', 'look', 'well', 'could', 'return', 'home']]


In [57]:
#guardando diccionario
with open(r"../data/interim/general_dictionary.pkl", "wb") as output_file:
    pickle.dump(id2word, output_file)

In [56]:
#guardando lista de textos tokenizada
with open(r"../data/interim/tokenized_words.pkl", "wb") as output_file:
    pickle.dump(data_lemmatized, output_file)


In [55]:
for key, value in id2word.items():
    print(key, value)

0 back
1 bottle
2 box
3 catering
4 clean
5 could
6 definitely
7 else
8 ever
9 family
10 feel
11 give
12 glove
13 good
14 hand
15 have
16 high
17 home
18 hope
19 hotel
20 impress
21 incredible
22 keep
23 look
24 mask
25 measure
26 money
27 much
28 need
29 offer
30 pandemic
31 place
32 price
33 put
34 recommend
35 reopen
36 return
37 room
38 safe
39 sanitiser
40 staff
41 standard
42 stay
43 thank
44 time
45 travel
46 upon_arrival
47 value
48 wear
49 week
50 well
51 would
52 amazing
53 appoint
54 area
55 arrive
56 awesome
57 background
58 bloom
59 blue
60 bougainvillea
61 building
62 business
63 center
64 church
65 city
66 comfortable
67 day
68 depart
69 dining
70 direction
71 estimate
72 experience
73 extensively
74 flower_arrangement
75 flowering_plant
76 foot
77 fresh
78 full
79 ground
80 height
81 large
82 make
83 manicure
84 minute
85 mountain
86 perfection
87 pleasure
88 preserve
89 reception
90 spectacular
91 truly
92 view
93 welcome
94 camera
95 container
96 exactly
97 fireplace
9

1085 indicate
1086 layout
1087 maintain
1088 touch
1089 afternoon
1090 buffet
1091 flight
1092 issue
1093 mine
1094 miss
1095 none
1096 second
1097 sister
1098 hospitality
1099 orient
1100 surrounding
1101 boutique
1102 casa_encantada
1103 decorate
1104 possibly
1105 relaxing
1106 soak
1107 avid
1108 phenomenal
1109 fabulous
1110 latin_america
1111 superb
1112 attractive
1113 beam
1114 begin
1115 build
1116 cement
1117 complimentary
1118 cushioned
1119 drinking
1120 efficient
1121 enchant
1122 expose
1123 flavor
1124 guatemalan
1125 ice
1126 intimate
1127 plunge
1128 robe
1129 studio
1130 sunbed
1131 sunny
1132 tea
1133 white
1134 wooden
1135 solution
1136 wherever
1137 daily
1138 inspire
1139 least
1140 occasional
1141 oversized
1142 pitcher
1143 refresh
1144 retreat
1145 station
1146 tub
1147 twice
1148 character
1149 sunrise
1150 taddition
1151 venue
1152 facility
1153 atmosphere
1154 bond
1155 collaborate
1156 event
1157 hold
1158 invite
1159 participant
1160 pensativo
1161 worksho

1874 valley
1875 packed
1876 spaciousness
1877 checked
1878 online
1879 opening
1880 security_guard
1881 zone
1882 tackle
1883 transfer
1884 aeration
1885 corresponding
1886 drain
1887 invade
1888 possibility
1889 secondary
1890 frappe
1891 overcome
1892 candle
1893 cinder
1894 wise
1895 equidistant
1896 extraordinarily
1897 fold
1898 obliging
1899 cachet
1900 dead
1901 improved
1902 knowledge
1903 rustic
1904 soon
1905 perfict
1906 slight
1907 splurge
1908 spotless
1909 unresponsive
1910 via_email
1911 heartbeat
1912 midst
1913 restful
1914 spoiled
1915 choosy
1916 gift
1917 peep
1918 craft
1919 degree
1920 plentiful
1921 premise
1922 refurbish
1923 accessorize
1924 speed
1925 break
1926 cortyard
1927 halfway
1928 hire
1929 offerede
1930 shape
1931 greeting
1932 organizing
1933 outing
1934 reschedule
1935 hurt
1936 public_area
1937 advantage
1938 expense
1939 football
1940 inexpensive
1941 nonprofit
1942 organization
1943 over
1944 refrigerated
1945 socialize
1946 prime
1947 absolute


2874 mad
2875 mat
2876 hint
2877 memo
2878 piano
2879 pace
2880 contrast
2881 crazy
2882 familiar
2883 herbal
2884 require
2885 throughout
2886 choco
2887 ham
2888 strawberry
2889 sugary
2890 appropriate
2891 especial
2892 fabulosa
2893 bucket
2894 chicken_bus
2895 consistently
2896 cram
2897 heap
2898 onsite
2899 three_monkey
2900 trickle
2901 ca
2902 fraise
2903 optimistic
2904 promote
2905 auntentical
2906 relate
2907 washroom
2908 gap
2909 reasonably
2910 tick
2911 trendy
2912 exelect
2913 extreamly
2914 definately
2915 humble
2916 grocery
2917 longer
2918 ur
2919 interaction
2920 astounding
2921 coin
2922 intelligent
2923 breed
2924 carry
2925 result
2926 scratch
2927 soup
2928 worie
2929 pax
2930 respectable
2931 anticipated
2932 dissapointe
2933 recomended
2934 equipment
2935 quesadilla
2936 talent
2937 treasure
2938 tuesday
2939 authorization
2940 domestic
2941 faulty
2942 heater
2943 ounce
2944 rear
2945 seek
2946 uncreative
2947 untrained
2948 parfait
2949 moreover
2950 playf

3873 abound
3874 category
3875 enjoyably
3876 bumpy
3877 cloister
3878 tuktuk
3879 chimaltenago
3880 sangria
3881 shoe
3882 sleepy
3883 custom
3884 pump
3885 scent
3886 date
3887 mid
3888 cent
3889 epitome
3890 intimidate
3891 misleading
3892 unkept
3893 socialise
3894 criterion
3895 cloroxed
3896 oriented
3897 proof
3898 scare
3899 soooooo
3900 dusty
3901 airbnbd
3902 delivery
3903 sparsely
3904 apply
3905 basement
3906 noticed
3907 deposit
3908 respiratory
3909 but
3910 ifs
3911 inventory
3912 pocket
3913 evrything
3914 used
3915 clarity
3916 explicitly
3917 incur
3918 quinta_de
3919 upfront
3920 hunch
3921 internet_connection
3922 remodel
3923 cooperative
3924 elderly
3925 entirely
3926 plus
3927 unfriendly
3928 avocado
3929 crab
3930 cuisine
3931 dinningroom
3932 feast
3933 midday
3934 personalbe
3935 shrimp
3936 waiter
3937 dependent
3938 mayan
3939 substantial
3940 smoothie
3941 retain
3942 booking_com
3943 rag
3944 sack
3945 runaway
3946 angel
3947 cable_tv
3948 dumpy
3949 assig

...

6623 entryway
6624 eternal
6625 wilderness
6626 protien
6627 magnet
6628 refined
6629 vault
6630 cappucino
6631 everyplace
6632 installation
6633 enthused
6634 kitschy
6635 elaborate
6636 seal
6637 conflict
6638 taint
6639 faced
6640 ful
6641 chame
6642 hair_dryer
6643 specification
6644 marible
6645 outfitter
6646 abode
6647 elses
6648 environ
6649 homage
6650 impede
6651 regrettably
6652 stargazer
6653 thunder
6654 watering
6655 wiifi
6656 ambassador
6657 nickel
6658 restaraunt
6659 plague
6660 seduce
6661 carnage
6662 evaluate
6663 atv
6664 astonished
6665 poke
6666 swap
6667 conside
6668 material
6669 niche
6670 sleape
6671 crutcher
6672 reeeealy
6673 internship
6674 serenade
6675 noted
6676 acknowledgment
6677 breathing
6678 slap
6679 positively
6680 command
6681 pie
6682 enforcement
6683 maneuver
6684 structured
6685 atrium
6686 tin
6687 prepaid
6688 transplant
6689 reverse
6690 dominate
6691 primitive
6692 retirement
6693 immerse
6694 rested
6695 usualy
6696 cobblestone_road
669

8230 potatoes_tomatoe
8231 mozzarella
8232 chuck
8233 ordinance
8234 promo
8235 sensational
8236 upstair
8237 backfire
8238 lens
8239 paranoia
8240 attiude
8241 enviorement
8242 sooooooooooo
8243 citizen
8244 problematic
8245 trained
8246 grid
8247 hipe
8248 daunted
8249 condense
8250 flowery
8251 horizon
8252 hunapu
8253 medley
8254 merge
8255 birds
8256 butterfly
8257 raunchy
8258 unpolice
8259 determine
8260 bedded
8261 inglish
8262 loot
8263 yoga_studio
8264 outweighs
8265 treehouse
8266 deliicous
8267 iliyana
8268 allowing
8269 mildly
8270 reccommende
8271 arrevie
8272 diagonal
8273 meantime
8274 roon
8275 reiterate
8276 experimentation
8277 liner
8278 trait
8279 gal
8280 anjeline
8281 cork
8282 hector
8283 mourning
8284 environmental
8285 frantically
8286 multicultural
8287 gay
8288 centerpiece
8289 bohemian
8290 execeede
8291 reccomendation
8292 scrape
8293 distinctive
8294 transorte
8295 oddball
8296 earth_lodge
8297 incline
8298 supposed
8299 hotelier
8300 compe
8301 congregat

9341 cess
9342 disgustingly
9343 mommy
9344 roundtrip
9345 distueb
9346 continent
9347 footed
9348 nomadic
9349 chundere
9350 overbearing
9351 cornflake
9352 cucumber
9353 unhygenic
9354 following
9355 nightstand
9356 impecably
9357 accordance
9358 disrepair
9359 tangle
9360 evety
9361 sorely
9362 overal
9363 belch
9364 air_circulation
9365 substitute
9366 casually
9367 durable
9368 embarrassing
9369 legitimate
9370 scrutinize
9371 sandwitche
9372 wage
9373 earthlodge
9374 smellyness
9375 extremeclimbe
9376 recommene
9377 baking
9378 beehive
9379 inspite
9380 math
9381 sweetly
9382 mornig
9383 minus
9384 exemplify
9385 grouse
9386 multinational
9387 correspondance
9388 egregiously
9389 pathetic
9390 prearrange
9391 prearranged
9392 increasingly
9393 conspire
9394 slate
9395 economically
9396 shadow
9397 menagerie
9398 involvement
9399 popularity
9400 verbal
9401 costly
9402 terrrace
9403 scheduling
9404 spreading
9405 waterbottle
9406 levely
9407 creditcard
9408 supportiv
9409 dilemma


10372 maim
10373 medic
10374 compelling
10375 compress
10376 b
10377 bestie
10378 tranport
10379 roster
10380 frisbee
10381 omnivore
10382 straightforward
10383 brutalist
10384 contribution
10385 raindrop
10386 donate
10387 troublesome
10388 mosquitos
10389 bougainvilla
10390 overhang
10391 tribute
10392 perfcet
10393 toxic
10394 whatnot
10395 unmet
10396 tabasco
10397 magician
10398 transporting
10399 diferent
10400 anmbience
10401 devotion
10402 governor
10403 society
10404 unequal
10405 knockout
10406 turnback
10407 yelow
10408 exclamation
10409 mary_sue
10410 stile
10411 evocative
10412 without_hesitation
10413 fifty
10414 blister
10415 compost
10416 credential
10417 hiker
10418 inaccessible
10419 musk
10420 dissapoint
10421 fantasy
10422 avaiable
10423 serenely
10424 unequalled
10425 guava
10426 founder
10427 guatemalait
10428 brace
10429 deluge
10430 overlap
10431 smal
10432 recipie
10433 varsity
10434 golden
10435 operative
10436 roberta
10437 pseudo
10438 noisier
10439 firewall

11371 zig
11372 soleil
11373 wrinkle
11374 prominently
11375 fullly
11376 dirge
11377 ethereal
11378 hallowed
11379 solemn
11380 grandpa
11381 ole
11382 timey
11383 broadcast
11384 exfoliation
11385 ordeal
11386 orphanage
11387 conversant
11388 uncomfrotable
11389 montior
11390 relic
11391 aborigine
11392 nitpicky
11393 occupancy
11394 recoop
11395 sofabed
11396 chiminea
11397 padding
11398 paisagism
11399 unbeliaveble
11400 brushed
11401 audiovisual
11402 cardholder
11403 configured
11404 botanical
11405 emersion
11406 hilight
11407 restruant
11408 giveaway
11409 jetlagged
11410 archeological
11411 niceamenitie
11412 wlke
11413 deficiency
11414 underdelivering
11415 outwards
11416 cardamom
11417 cornmeal
11418 questionnaire
11419 ballroom
11420 caulking
11421 gifted
11422 wll
11423 stacked
11424 bellboy
11425 individuality
11426 unattended
11427 splendorous
11428 warmness
11429 insurance
11430 actully
11431 deodorizer
11432 especillay
11433 lightup
11434 definitive
11435 timeline
1143

In [35]:
# Create Corpus
#guardar
texts = data_lemmatized

# Term Document Frequency
#guardar
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[1:2])

[[(4, 1), (8, 1), (10, 1), (13, 1), (19, 1), (37, 1), (40, 1), (42, 1), (45, 1), (50, 1), (52, 1), (53, 1), (54, 3), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1)]]


In [36]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('back', 1),
  ('bottle', 1),
  ('box', 1),
  ('catering', 1),
  ('clean', 1),
  ('could', 1),
  ('definitely', 1),
  ('else', 1),
  ('ever', 1),
  ('family', 1),
  ('feel', 1),
  ('give', 1),
  ('glove', 2),
  ('good', 1),
  ('hand', 1),
  ('have', 2),
  ('high', 1),
  ('home', 1),
  ('hope', 1),
  ('hotel', 2),
  ('impress', 1),
  ('incredible', 2),
  ('keep', 1),
  ('look', 1),
  ('mask', 2),
  ('measure', 1),
  ('money', 1),
  ('much', 1),
  ('need', 1),
  ('offer', 1),
  ('pandemic', 1),
  ('place', 1),
  ('price', 1),
  ('put', 1),
  ('recommend', 1),
  ('reopen', 1),
  ('return', 1),
  ('room', 1),
  ('safe', 1),
  ('sanitiser', 1),
  ('staff', 2),
  ('standard', 1),
  ('stay', 2),
  ('thank', 1),
  ('time', 1),
  ('travel', 1),
  ('upon_arrival', 1),
  ('value', 1),
  ('wear', 1),
  ('week', 1),
  ('well', 2),
  ('would', 1)]]

## Latent Dirichlet Allocation

![Latent Dirichlet Allocation](https://miro.medium.com/max/1400/1*fCc0JT3W-1ViYyw0hJ7rdA.jpeg)

In [39]:
#entrenando el modelo
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
#guardar modelo
with open(r"../models/model.pkl", "wb") as output_file:
    pickle.dump(lda_model, output_file)

In [40]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())

[(0,
  '0.123*"set" + 0.105*"worth" + 0.076*"overall" + 0.063*"walking_distance" + '
  '0.049*"attention" + 0.045*"centre" + 0.043*"do" + 0.036*"traveler" + '
  '0.033*"other" + 0.033*"expectation"'),
 (1,
  '0.187*"quite" + 0.150*"space" + 0.086*"manager" + 0.076*"change" + '
  '0.052*"access" + 0.046*"convenient" + 0.038*"store" + 0.036*"employee" + '
  '0.032*"complain" + 0.031*"information"'),
 (2,
  '0.196*"bar" + 0.082*"think" + 0.073*"wedding" + 0.060*"dorm" + 0.041*"loud" '
  '+ 0.041*"cool" + 0.040*"hammock" + 0.037*"money" + 0.034*"busy" + '
  '0.030*"complaint"'),
 (3,
  '0.217*"service" + 0.190*"food" + 0.052*"eat" + 0.040*"world" + '
  '0.038*"drink" + 0.038*"site" + 0.034*"menu" + 0.032*"order" + '
  '0.030*"history" + 0.025*"cook"'),
 (4,
  '0.107*"breakfast" + 0.046*"small" + 0.041*"even" + 0.037*"shower" + '
  '0.034*"water" + 0.032*"hot" + 0.028*"morning" + 0.027*"first" + '
  '0.026*"coffee" + 0.026*"trip"'),
 (5,
  '0.092*"check" + 0.088*"pay" + 0.082*"ask" + 0.070*

In [41]:
topics = (lda_model.print_topics())

In [42]:
topics

[(0,
  '0.123*"set" + 0.105*"worth" + 0.076*"overall" + 0.063*"walking_distance" + 0.049*"attention" + 0.045*"centre" + 0.043*"do" + 0.036*"traveler" + 0.033*"other" + 0.033*"expectation"'),
 (1,
  '0.187*"quite" + 0.150*"space" + 0.086*"manager" + 0.076*"change" + 0.052*"access" + 0.046*"convenient" + 0.038*"store" + 0.036*"employee" + 0.032*"complain" + 0.031*"information"'),
 (2,
  '0.196*"bar" + 0.082*"think" + 0.073*"wedding" + 0.060*"dorm" + 0.041*"loud" + 0.041*"cool" + 0.040*"hammock" + 0.037*"money" + 0.034*"busy" + 0.030*"complaint"'),
 (3,
  '0.217*"service" + 0.190*"food" + 0.052*"eat" + 0.040*"world" + 0.038*"drink" + 0.038*"site" + 0.034*"menu" + 0.032*"order" + 0.030*"history" + 0.025*"cook"'),
 (4,
  '0.107*"breakfast" + 0.046*"small" + 0.041*"even" + 0.037*"shower" + 0.034*"water" + 0.032*"hot" + 0.028*"morning" + 0.027*"first" + 0.026*"coffee" + 0.026*"trip"'),
 (5,
  '0.092*"check" + 0.088*"pay" + 0.082*"ask" + 0.070*"try" + 0.056*"charge" + 0.056*"end" + 0.052*"tell

In [43]:
doc_lda = lda_model[corpus]

In [44]:
#Perplejidad
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Score de coherencia
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.87748149460469

Coherence Score:  0.30472386296584775


In [45]:
# Visualizamos los temas
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis
#Grabar visualizacion
pyLDAvis.save_html(vis,'../src/visualization/output.html')

In [46]:
#PREDICTION
#creando nuevas samples data
sample_data = ['Called in here for dinner. Great little place only sits about 8 and you can watch the chef cook behind the bar. We tried the gringo and spicy Italian with orange & fries. Both were excellent and great value for money',
               'We stayed at San Rafael for 3 night in March 2020. Our room was great and the service was amazing!!! They suggested wonderful restaurants ans a great walking tour to start off our trip... The hotel grounds are private and beautiful perfectly located within Antigua! The stay surpassed any expectations we had and highly recommend to anybody visiting Antigua!!!',
               'We had a great experience climbing Acatenango with CA Travelers. Everything is well-considered and thoughtfully curated to ensure that hikers will enjoy all aspects of the trip. We appreciated the warm sleeping bags, blankets and mattresses, the heavy jackets, the eco-friendly approach to the food, our guides. We felt safe and comfortable.And the little touches, such as hot cocoa and smoothies at check-out, are delightful and show the care the CA takes to create a great experience. We definitely recommend this company for anyone who is planning to climb Acatenango']

#sample_data = data[0:3]
#toeknizacion simple
sample_data_words = list(sent_to_words(sample_data))
#pasar sample_data a bow
# Eliminamos stopwords
sample_data_words_nostops = remove_stopwords(sample_data_words)
# Formamos bigrams
sample_data_words_bigrams = make_bigrams(sample_data_words_nostops)
#este ya cargó antes
#nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

# Lematizamos preservando únicamente noun, adj, vb, adv
sample_data_lemmatized = lemmatization(sample_data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Creamos diccionario
# NO VALIDO POR QUE EL DICCIONARIO QUE SE DEBE DE TOMAR ES EL DICCIONARIO ORIGINAL, EL DEL ENTRENAMIENTO
#s_id2word = corpora.Dictionary(sample_data_lemmatized)

In [47]:
sample_data

['Called in here for dinner. Great little place only sits about 8 and you can watch the chef cook behind the bar. We tried the gringo and spicy Italian with orange & fries. Both were excellent and great value for money',
 'We stayed at San Rafael for 3 night in March 2020. Our room was great and the service was amazing!!! They suggested wonderful restaurants ans a great walking tour to start off our trip... The hotel grounds are private and beautiful perfectly located within Antigua! The stay surpassed any expectations we had and highly recommend to anybody visiting Antigua!!!',
 'We had a great experience climbing Acatenango with CA Travelers. Everything is well-considered and thoughtfully curated to ensure that hikers will enjoy all aspects of the trip. We appreciated the warm sleeping bags, blankets and mattresses, the heavy jackets, the eco-friendly approach to the food, our guides. We felt safe and comfortable.And the little touches, such as hot cocoa and smoothies at check-out, a

In [48]:
print(sample_data_lemmatized[:1])


[['call', 'dinner', 'great', 'little', 'place', 'sit', 'watch', 'chef', 'cook', 'bar', 'try', 'gringo', 'spicy', 'italian', 'orange', 'fry', 'excellent', 'great', 'value', 'money']]


In [49]:
# Create Corpus de SAMPLE
sample_texts = sample_data_lemmatized

# Term Document Frequency de SAMPLE
# tomando de referencia eL diccionario del entrenamiento
sample_corpus = [id2word.doc2bow(text) for text in sample_texts]

# View
print(sample_corpus[1:2])

[[(19, 1), (34, 1), (37, 1), (42, 1), (52, 1), (79, 1), (102, 1), (123, 1), (138, 1), (141, 1), (178, 1), (222, 1), (229, 1), (233, 1), (256, 1), (357, 1), (508, 1), (603, 1), (610, 1), (788, 1), (822, 1), (1022, 1), (1023, 1), (1024, 1)]]


In [50]:
#encontrando EL TOPIC MAS REPRESENTATIVO
def format_topics_sentences(ldamodel=None, corpus=sample_corpus, texts=sample_texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[sample_corpus]):
    #for i, row_list in enumerate(sample_data):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=sample_corpus, texts=sample_data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,9.0,0.4831,"room, stay, place, staff, great, good, night, ...",Called in here for dinner. Great little place ...
1,1,9.0,0.4793,"room, stay, place, staff, great, good, night, ...",We stayed at San Rafael for 3 night in March 2...
2,2,9.0,0.4980,"room, stay, place, staff, great, good, night, ...",We had a great experience climbing Acatenango ...


In [51]:
topic_names = [
[0,'travelers'],
[1,'mindfullness'],
[2, 'party_and_fun'],
[3, 'food_service'],
[4, 'morning_food'],
[5, 'money_value'],
[6, 'after_hour_service'],
[7, 'friendship'],
[8, 'pool_and_spa'],
[9, 'beautiful_room'],
[10, 'hostel'],
[11, 'beatiful_experience'],
[12, 'poor_quality'],
[13, 'comfy'],
[14, 'poor_room'],
[15, 'illness'],
[16, 'garden'],
[17, 'extra_services'],
[18, 'poor_experience'],
[19, 'restaurant_area']
]

In [52]:
df_topics = pd.DataFrame(topic_names, columns = ['id_topic', 'name'])  
df_topics

,id_topic,name
0,0,travelers
1,1,mindfullness
2,2,party_and_fun
3,3,food_service
4,4,morning_food
5,5,money_value
6,6,after_hour_service
7,7,friendship
8,8,pool_and_spa
9,9,beautiful_room


In [53]:
df_dominant_topic

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,9.0,0.4831,"room, stay, place, staff, great, good, night, ...",Called in here for dinner. Great little place ...
1,1,9.0,0.4793,"room, stay, place, staff, great, good, night, ...",We stayed at San Rafael for 3 night in March 2...
2,2,9.0,0.4980,"room, stay, place, staff, great, good, night, ...",We had a great experience climbing Acatenango ...


In [54]:
df_final = pd.merge(left=df_dominant_topic, right=df_topics, how='left', left_on='Dominant_Topic', right_on='id_topic')
df_final

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,id_topic,name
0,0,9.0,0.4831,"room, stay, place, staff, great, good, night, ...",Called in here for dinner. Great little place ...,9,beautiful_room
1,1,9.0,0.4793,"room, stay, place, staff, great, good, night, ...",We stayed at San Rafael for 3 night in March 2...,9,beautiful_room
2,2,9.0,0.4980,"room, stay, place, staff, great, good, night, ...",We had a great experience climbing Acatenango ...,9,beautiful_room


In [74]:
df_final.iloc[:, [0,1,6,4]].values.tolist()

[[0,
  9.0,
  'beautiful_room',
  'Called in here for dinner. Great little place only sits about 8 and you can watch the chef cook behind the bar. We tried the gringo and spicy Italian with orange & fries. Both were excellent and great value for money'],
 [1,
  9.0,
  'beautiful_room',
  'We stayed at San Rafael for 3 night in March 2020. Our room was great and the service was amazing!!! They suggested wonderful restaurants ans a great walking tour to start off our trip... The hotel grounds are private and beautiful perfectly located within Antigua! The stay surpassed any expectations we had and highly recommend to anybody visiting Antigua!!!'],
 [2,
  9.0,
  'beautiful_room',
  'We had a great experience climbing Acatenango with CA Travelers. Everything is well-considered and thoughtfully curated to ensure that hikers will enjoy all aspects of the trip. We appreciated the warm sleeping bags, blankets and mattresses, the heavy jackets, the eco-friendly approach to the food, our guides.